## Podstawy Uczenia Maszynowego – Projekt
### Rozpoznawanie nadużyć w transakcjach płatniczych na podstawie fizycznych okoliczności ich wykonania
#### Autorzy: Kacper Korecki, Dominik Zakrzewski, Konrad Zbylut

## Wprowadzenie

Celem naszego projektu jest analiza obszernego zbioru transakcji płatniczych za pomocą kilku wybranych algorytmów dla porównania ich charakterystyki i otrzymanych wyników.

## O zbiorze

Prezentowany przez nas zbiór danych "card transaction data" jest zbiorem przedstawiającym transakcje dokonywane kartami płatniczymi. Ilość rekordów w datasecie wynosi 1 000 000.

Argumenty:
* `distance_from_home` – odległość miejsca wykonania transakcji od adresu domowego klienta banku,
* `distance_from_last_transaction` – odległość miejsca wykonania transakcji od miejsca poprzedniej transakcji,
* `ratio_to_median_purchase_price` – stosunek wysokości kwoty transakcji do mediany wszystkich transakcji danego klienta,
* `repeat_retailer` – argument logiczny (0.0/1.0) mówiący o tym, czy dany klient wykonywał już transakcję u danego sprzedającego,
* `used_chip` – argument logiczny (0.0/1.0) mówiący o tym, czy przy transakcji została użyta karta (tj. czy faktycznie plastikowa karta została włożona np. do terminalu)
* `used_pin_number` – argument logiczny (0.0/1.0) mówiący o tym, czy podczas transakcji został podany numer PIN,
* `online_order` – argument logiczny (0.0/1.0) mówiący o tym, czy transakcja została wykonana przez Internet,

Wynik:
* `fraud` – argument logiczny (0.0/1.0) mówiący o tym, czy transakcja była nadużyciem (czyli np. została wykonana przez osobę, która włamała się na konto)

## Wczytanie i obróbka zbioru

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

Wczytujemy zbiór danych z pliku CSV.

In [ ]:
data = pd.read_csv('card_transdata.csv', delimiter=',')

In [ ]:
df = pd.DataFrame(data)
df.describe()

Ponadto sprawdzamy stosunek transakcji oznaczonych jako nadużycia (y=1.0) do liczby wszystkich transakcji w zbiorze

In [ ]:
data_np = df.to_numpy()

In [ ]:
total_fraud_transactions = np.sum(data_np[:, 7])
print("Percentage of fraud transactions:", total_fraud_transactions / data_np.shape[0] * 100, "%")

Jak widzimy wyniki nie są zbyt "różnorodne", gdyż większość transakcji stanowią transakcje non-fraud. Ponadto liczba rekordów jest bardzo duża, więc na potrzeby prezentacji i ze względu na złożoność niektórych algorytmów (np. SVM) musimy zmniejszyć nieco liczbę rekordów. Stąd też przyszedł pomysł eksperymentu dostosowania zbioru tak by liczba transakcji "normalnych" była ograniczona tj. podobna do liczby transakcji oznaczonych jako nadużycia, a więc łącznie dostaniemy 160 000 transakcji.

In [ ]:
X_raw = data_np[:, :-1]
y_raw = data_np[:, -1]

In [ ]:
#from sklearn.preprocessing import StandardScaler
#X_raw = StandardScaler().fit_transform(X_raw)

In [ ]:
X = []
y = []

non_fraud_limit = 85000
non_fraud_c = 0

for i in range(len(X_raw)):
    if y_raw[i] == 0.0 and non_fraud_c <= non_fraud_limit:
        X.append(X_raw[i])
        y.append(y_raw[i])
        non_fraud_c += 1
    elif y_raw[i] == 1.0:
        X.append(X_raw[i])
        y.append(y_raw[i])

X = np.array(X)
y = np.array(y)

Dokonujemy podziału na zbiory X i y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.3, random_state=1)

A także przygotowujemy funkcję pomocniczą przy rysowaniu macierzy pomyłek:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

def confusion_matrix_from_preds(model, title):
    preds = model.predict(X_test)
    cm = ConfusionMatrixDisplay.from_predictions(y_test, preds)
    cm.ax_.set_title(title)
    plt.show()
    print(classification_report(y_test, preds))

## Naiwny klasyfikator Bayesa

In [ ]:
from sklearn.naive_bayes import GaussianNB, ComplementNB, BernoulliNB, MultinomialNB
import warnings
warnings.filterwarnings('ignore')

In [ ]:
gaussian_nb_model = GaussianNB().fit(X_train, y_train)

gaussian_nb_train_sc = gaussian_nb_model.score(X_train, y_train)
gaussian_nb_test_sc = gaussian_nb_model.score(X_test, y_test)

confusion_matrix_from_preds(gaussian_nb_model, 'Gaussian NB Confusion Matrix')

In [ ]:
complement_nb_model = ComplementNB().fit(X_train, y_train)

complement_nb_train_sc = complement_nb_model.score(X_train, y_train)
complement_nb_test_sc = complement_nb_model.score(X_test, y_test)

confusion_matrix_from_preds(complement_nb_model, 'Complement NB Confusion Matrix')

In [ ]:
bernoulli_nb_model = BernoulliNB().fit(X_train, y_train)

bernoulli_nb_train_sc = bernoulli_nb_model.score(X_train, y_train)
bernoulli_nb_test_sc = bernoulli_nb_model.score(X_test, y_test)

confusion_matrix_from_preds(bernoulli_nb_model, 'Bernoulli NB Confusion Matrix')

In [ ]:
multinomial_nb_model = MultinomialNB().fit(X_train, y_train)

multinomial_nb_train_sc = multinomial_nb_model.score(X_train, y_train)
multinomial_nb_test_sc = multinomial_nb_model.score(X_test, y_test)

confusion_matrix_from_preds(multinomial_nb_model, 'Multinomial NB Confusion Matrix')

## SVM

In [ ]:
from sklearn import svm

In [ ]:
svm_model = svm.SVC(kernel='rbf', verbose=True).fit(X_train, y_train)
preds = svm_model.predict(X_test)

In [ ]:
confusion_matrix_from_preds(svm_model, 'SVM Confusion Matrix')

### SVM with RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
svm_model = svm.SVC( verbose=True)
params = {
    'kernel': ['rbf', 'poly'],
    'C': [1, 10, 100, 1000]
}

rs = RandomizedSearchCV(svm_model, params, cv=3, verbose=2)
rs.fit(X_train, y_train)

In [ ]:
confusion_matrix_from_preds(rs, 'SVM with RS Confusion Matrix')

In [ ]:
print(f'Najlepsze parametry: {rs.best_params_}')

Niestety algorytm ten dla dużej ilości danych w naszym przypadku się nie sprawdzi, gdyż jego złożoność obliczeniowa oznacza wiele godzin oczekiwania na wynik, który możemy osiągnąć innymi metodami.

## Drzewa decyzyjne

In [ ]:
from sklearn import tree

In [ ]:
dt_classifier = tree.DecisionTreeClassifier(random_state=1)
dt_classifier.fit(X_train, y_train)

Wyświetlamy głębokość drzewa, dla której klasyfikator osiąga najlepsze wyniki

In [ ]:
print(dt_classifier.get_depth())

Ponadto wyświetlamy tablicę feature importances, czyli istotności poszczególnych argumentów dla działania modelu

In [ ]:
print(dt_classifier.feature_importances_)

Chcielibyśmy też zobaczyć wykres przedstawiający zależność głębokości od osiąganych wyników

In [ ]:
classifiers, scores_train, scores_test = [], [], []
depths = np.arange(2, 15)

for depth in depths:
    classifier = tree.DecisionTreeClassifier(random_state=0, max_depth=depth)
    classifier.fit(X_train, y_train)
    classifiers.append(classifier)

    scores_train.append(classifier.score(X_train, y_train))
    scores_test.append(classifier.score(X_test, y_test))

In [ ]:
plt.plot(depths, scores_train, c='b', label='train')
plt.plot(depths, scores_test, c='r', label='test')
plt.legend()
plt.show()
best = 2 + np.argmax(scores_test)
print(best)

Interesuje nas też rozkład poszczególnych przypadków testowych, dlatego rysujemy macierz pomyłek

In [ ]:
confusion_matrix_from_preds(dt_classifier, 'Decision Tree Classifier Confusion Matrix')

## Analiza najlepszego modelu

Po wybraniu najlepszego modelu:

In [ ]:
best_classifier = dt_classifier

Możemy przejść do jego analizy, np. poprzez ręczne symulowanie transakcji i sprawdzanie wyników. W tym celu wykorzystamy widżety jupytera.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(distance_from_home, distance_from_last_transaction, ratio_to_median_purchase_price, repeat_retailer, used_chip, used_pin_number, online_order):
    res = best_classifier.predict(np.array([[distance_from_home, distance_from_last_transaction, ratio_to_median_purchase_price, repeat_retailer, used_chip, used_pin_number, online_order]]))
    y = res[0]
    if y == 0.0:
        return "Transaction is not a fraud"
    else:
        return "Transaction is FRAUD"

In [ ]:
w = interact(f, distance_from_home=widgets.FloatSlider(min=0.0, max=500.0, step=1.0), distance_from_last_transaction=widgets.FloatSlider(min=0.0, max=500.0, step=1.0), ratio_to_median_purchase_price=widgets.FloatSlider(min=0.0, max=10.0, step=0.1), repeat_retailer=True, used_chip=False, used_pin_number=False, online_order=False)

### Podgląd drzewa

Ponadto algorytm drzew decyzyjnych jest algorytmem "white box", czyli takim, którego działanie możemy prześledzić. W tym celu wydrukujemy sobie drzewo do pliku graficznego.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(30, 10), dpi=600)
tree.plot_tree(best_classifier)

plt.show(fig)

### Test wszystkiego

Na koniec możemy także sprawdzić dokładność działania modelu na całym zbiorze, który posiadamy:

In [ ]:
best_classifier.score(X_raw, y_raw)